### import required libraries


In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel,Trainer, TrainingArguments, TrOCRForCausalLM
from PIL import Image
import torch
import glob
import cv2
import numpy as np
import pandas as pd 
from tkinter import filedialog
import fitz
import os
from natsort import natsorted
from collections import OrderedDict

In [2]:
device = torch.device("cpu") #"cuda" if torch.cuda.is_available() else 

In [3]:
# processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
# model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

In [4]:
def get_images_from_folder(folder_path):
    files = os.listdir(folder_path)
    image_files = [os.path.join(folder_path, file) for file in files if file.endswith(('.png', '.jpg', '.jpeg'))]
    
    return image_files


In [5]:
def ocr(image, processor, model):
    image = Image.open(image).convert('RGB')
    pixel_values = processor(image, return_tensors='pt').pixel_values.to(device)
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return generated_text

### training a model

In [6]:
# from datasets import Dataset
# import pandas as pd
# # df = pd.read_csv(r'D:\MAIN\MEMORY_STACK\scanToText\TROCR\formData copy.csv', header=None, names=["image_path", "text"])
# df = pd.read_csv(r'D:\MAIN\MEMORY_STACK\scanToText\TROCR\final_dataset.csv', header=None, names=["image_path", "text"])

# # Convert the DataFrame into a Hugging Face Dataset
# dataset = Dataset.from_pandas(df)

# print(df.columns)  # Ensure "image_path" and "text" or their equivalents exist
# print(df.head())   # Inspect a few rows to confirm structure


In [7]:

# # Define your preprocessing function
# def preprocess_function(examples):
#     # Load images from the "image_path" column
#     images = [Image.open(path).convert("RGB") for path in examples["image_path"]]
    
#     # Convert images to pixel values using the processor
#     pixel_values = processor(images=images, return_tensors="pt").pixel_values
#     print(pixel_values.shape)
#     # Tokenize the text labels
#     labels = processor.tokenizer(examples["text"], return_tensors="pt",max_length=7, padding=True, truncation=True).input_ids
    
#     return {"pixel_values": pixel_values, "labels": labels}

# # Apply the preprocessing function to the dataset
# # train_dataset = dataset.shuffle(seed=42).map(preprocess_function, batched=True)
# train_dataset = dataset.map(preprocess_function, batched=True)


In [8]:
# model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
# model.config.pad_token_id = processor.tokenizer.pad_token_id


In [9]:

# # training_args = TrainingArguments(
# #     output_dir="./results",
# #     learning_rate=2e-5,
# #     per_device_train_batch_size=16,
# #     num_train_epochs=3,
# # )


# # from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer

# # training_args = Seq2SeqTrainingArguments(
# #     output_dir="./results",
# #     eval_strategy="no",
# #     per_device_train_batch_size=8,
# #     per_device_eval_batch_size=8,
# #     gradient_accumulation_steps= 4,
# #     predict_with_generate=True,
# #     logging_steps=10,
# #     num_train_epochs=3,
# #     fp16=True,
# #     use_cpu=True
# # )

# from transformers import TrainingArguments, Trainer

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     per_device_train_batch_size=8,
#     # per_device_eval_batch_size=8,
#     evaluation_strategy="no",
#     num_train_epochs=3,
#     save_steps=500,
#     # eval_steps=500,
#     logging_dir="./logs",
#     learning_rate=5e-5
# )


In [10]:

# # trainer = Trainer(
# #     model=model,
# #     args=training_args,
# #     train_dataset=dataset
# # )

# # trainer = Seq2SeqTrainer(
# #     model=model.to(device),
# #     args=training_args,
# #     train_dataset= train_dataset
# # )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,  # Your training dataset
#     # eval_dataset=eval_dataset     # Your evaluation dataset
# )


In [11]:

# # Train the model
# trainer.train()


In [12]:
# model.save_pretrained("TROCR_final_dataset0/fine-tuned-model")
# processor.save_pretrained("TROCR_final_dataset0/fine-tuned-processor")

### using the trained model

In [13]:
model = VisionEncoderDecoderModel.from_pretrained("TROCR_final_dataset0/fine-tuned-model").to(device)
processor = TrOCRProcessor.from_pretrained("TROCR_final_dataset0/fine-tuned-processor")

### Table Detection

In [14]:


filename=filedialog.askopenfilename(initialdir=r'D:\MAIN\MEMORY_STACK\scanToText\images\random\test\test_new',title="select a file",filetypes=(("pdf files","*.jpeg *.png *.jpg *.pdf"),("All files", "*.*"))) #select the file 

def pdf_to_images(pdf_path, output_folder): #convert pdf to image
    document = fitz.open(pdf_path)
    for page_number in range(len(document)): #select each page of the pdf
        page = document.load_page(page_number)
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        image_name = f"{output_folder}/data{page_number + 1}.jpg"
        img.save(image_name)

    document.close()

    return image_name

def preprocess_image(filename):
    if filename.lower().endswith('.pdf'):
        filename= (pdf_to_images(filename,r'D:\handwritten_recognition\pdf_to_image'))
    elif filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        filename= (filename)
    else:
        print("Unsupported file type.")
    return filename

image_path = preprocess_image(filename)

In [15]:

# Function to delete all files in a folder
def clear_folder(folder_path):
    files = glob.glob(os.path.join(folder_path, '*')) # select each file from the folder 
    for file in files:
        try:
            os.remove(file) # removing all files before appending cropped cells in the folder
        except Exception as e:
            print(f"Error deleting {file}: {e}")

# Sort contours by row
def sort_contours_by_row(cnts, threshold=10):
    boundingBoxes = [cv2.boundingRect(c) for c in cnts] # use the bounding box detected by openCV
    cnts_boundingBoxes = sorted(zip(cnts, boundingBoxes), key=lambda b: b[1][1])  # Sort by y-coordinate
    rows = []
    current_row = []
    prev_y = cnts_boundingBoxes[0][1][1]  # Initial y-coordinate of the first bounding box
    
    for contour, bbox in cnts_boundingBoxes:
        x, y, w, h = bbox
        if abs(y - prev_y) <= threshold:
            current_row.append((contour, bbox))
        else:
            rows.append(current_row)
            current_row = [(contour, bbox)]
        prev_y = y
    if current_row:
        rows.append(current_row)
    sorted_cnts = []
    sorted_boundingBoxes = []
    for row in rows:
        row_sorted = sorted(row, key=lambda b: b[1][0])  # Sort by x-coordinate
        sorted_cnts.extend([c[0] for c in row_sorted])
        sorted_boundingBoxes.extend([c[1] for c in row_sorted])
    return sorted_cnts, sorted_boundingBoxes

# Function to rotate the image based on detected lines
def rotate_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi/180, 200)

    angles = []
    for line in lines:
        for rho, theta in line:
            angle = np.degrees(theta) - 90  # Convert radians to degrees and adjust the angle
            angles.append(angle)
    
    median_angle = np.median(angles)  # Compute median angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, median_angle, 1.0)
    rotated_image = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    
    return rotated_image

# detect the table and break the table into cells
def table_detection(img_path,folder_path):
    clear_folder(folder_path)
    img = cv2.imread(img_path)
    img = rotate_image(img)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    (_, thresh) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    kernel_length_v = (np.array(gray).shape[1])//60
    kernel_length_h = (np.array(gray).shape[1])//50
    
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,kernel_length_v )) 
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length_h, 1))
    
    im_temp1 = cv2.erode(thresh, vertical_kernel, iterations=3)
    im_temp2 = cv2.erode(thresh, horizontal_kernel, iterations=3)

    vertical_lines_img = cv2.dilate(im_temp1, vertical_kernel, iterations=3) # detect the vertical lines 
    horizontal_lines_img = cv2.dilate(im_temp2, horizontal_kernel, iterations=3) # detect the horizontal lines 

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    table_segment = cv2.addWeighted(vertical_lines_img, 0.5, horizontal_lines_img, 0.5, 0.0) # use the detected lines to form a table 
    table_segment = cv2.erode(cv2.bitwise_not(table_segment), kernel, iterations=2)
    _, table_segment = cv2.threshold(table_segment, 0, 255, cv2.THRESH_OTSU)
   
    contours, hierarchy = cv2.findContours(table_segment, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # detect different cells of the detected table
    contours = sort_contours_by_row(contours)[0] 
    count = 0
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if w > h and w > 15 and h > 10:
            count += 1
            cropped = img[y-2:y + h +5, x-2:x + w+3] #crop the detected cell and save it
            cell_filename = f'cell_{count}.png'
            cv2.imwrite(f'cropped_cells/{cell_filename}', cropped)
            cv2.rectangle(img,(x, y),(x + w, y + h),(0, 255, 0))

    if True:    
        cv2.imwrite("table.jpg", img)
        cv2.imwrite("table_detect.jpg", table_segment)
        
folder_path = r'D:\handwritten_recognition\cropped_cells' # folder where cropped cell with be saved
table_detection(image_path, folder_path)

In [16]:
def preprocess_image(image):
    image = cv2.imread(image) # load image using OpenCV
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (1, 1), 0)
    _, thresholded_image = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    denoised_image = cv2.fastNlMeansDenoising(thresholded_image, None, 30, 7, 21)
    processed_image_path = 'processed_image.jpg'
    cv2.imwrite(processed_image_path, thresholded_image)
    return processed_image_path


### use cropped cells for handwritten text detection

In [17]:
cell_images = get_images_from_folder(folder_path)
detected_text = {}
for idx, cell_path in enumerate(cell_images):
    processed_cell_path = preprocess_image(cell_path) # preprocessing each image 
    text = ocr(processed_cell_path, processor, model) # using trocr model to predict handwritten text
    detected_text[(idx)] = {'filename': os.path.basename(cell_path),'text': text} # add predicted text to a dictionary

c:\Users\tules\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [18]:
image_filenames = os.listdir(folder_path)
sorted_detected_text = OrderedDict(natsorted(detected_text.items(), key=lambda item: item[1]['filename'])) #sort dictionary items on the basis of filename 
print(sorted_detected_text)
print(len(sorted_detected_text))

OrderedDict({0: {'filename': 'cell_1.png', 'text': 'S.No.'}, 101: {'filename': 'cell_2.png', 'text': 'Plate ID'}, 112: {'filename': 'cell_3.png', 'text': 'Thick'}, 123: {'filename': 'cell_4.png', 'text': 'Width'}, 134: {'filename': 'cell_5.png', 'text': 'Length'}, 145: {'filename': 'cell_6.png', 'text': 'Unique ID'}, 156: {'filename': 'cell_7.png', 'text': 'Quality Remark'}, 167: {'filename': 'cell_8.png', 'text': '1'}, 178: {'filename': 'cell_9.png', 'text': '3419411ACC'}, 1: {'filename': 'cell_10.png', 'text': '10'}, 12: {'filename': 'cell_11.png', 'text': '2000'}, 23: {'filename': 'cell_12.png', 'text': '12000'}, 34: {'filename': 'cell_13.png', 'text': 'E250BR'}, 45: {'filename': 'cell_14.png', 'text': '04R'}, 56: {'filename': 'cell_15.png', 'text': '2'}, 67: {'filename': 'cell_16.png', 'text': '3419411ACE'}, 78: {'filename': 'cell_17.png', 'text': '10'}, 89: {'filename': 'cell_18.png', 'text': '2000'}, 100: {'filename': 'cell_19.png', 'text': '12000'}, 102: {'filename': 'cell_20.pn

### add the result to excel file

In [19]:
excel_name = os.path.splitext(os.path.basename(image_path))[0]
excel_file_path = fr'D:\handwritten_recognition\{excel_name}.xlsx' # Specify your desired file path

rows = []
row = []
i = 0
for key, value in sorted_detected_text.items():
    i+=1
    row.append(value['text']) # append different cells of a single row 
    print(f'{value['text']},',end=' ')
    if i%7==0:
        print()
        rows.append(row)  # append the collected row to rows
        row = [] 
# append any remaining items in the last row (if less than 7)
if row:
    rows.append(row)

# create DataFrame and save to Excel
df = pd.DataFrame(rows,columns=rows[0]) # add first row as column name
df = df[1:] # removing the first as we used it as column
df.reset_index(drop=True, inplace=True)
# save the DataFrame to Excel
df.to_excel(excel_file_path, index=False)

print(f'Data has been written to {excel_file_path}')

S.No., Plate ID, Thick, Width, Length, Unique ID, Quality Remark, 
1, 3419411ACC, 10, 2000, 12000, E250BR, 04R, 
2, 3419411ACE, 10, 2000, 12000, text, 04, 
3, 3419409CCB, 10, 2000, 12000, text, 04k, 
4, 3419409CCC, 10, 2000, 12000, text, 04R, 
5, 3419432DAO, 22, 2000, 12000, text, 01pcs, 
6, text, text, text, text, text, text, 
7, text, text, 1, text, text, text, 
6, text, text, 1, text, text, text, 
9, text, text, text, text, text, text, 
10, text, 70, total, total, text, text, 
11, text, text, text, text, text, text, 
12, text, text, text, text, text, text, 
13, text, text, text, text, text, text, 
14, text, text, text, text, text, text, 
15, text, text, text, text, text, text, 
16, text, text, text, text, text, text, 
17, 1, text, text, text, text, text, 
18, text, text, text, text, text, text, 
18, text, text, text, text, text, text, 
20, text, text, text, text, text, text, 
21, text, text, text, text, text, text, 
22, text, text, text, text, text, text, 
23, text, text, text, text